### 문제1: 246쪽 문제 6.16

<표 6.10>는 연구대상자들을 임의적으로 대조군과 처리군으로 배정하여 실험한 자료이다. 처리군에 배정된 연구대상자들은 사일리움(psyllium)을 포함하는 시리얼을 연구기간 동안 매일 일정량씩 섭취하였다. 이 연구의 목적은 이 시리얼이 LDL 콜레스테롤을 낮추는 효과가 있는지 알아보는 것이다. 빈도론적 방법이나 베이지안 방법을 사용하여 말기 콜레스테롤 수준에 대한 모형을 구하고자 한다. 초기의 콜레스테롤 수준은 (a) 적절한 점수를 사용하여 공변량(양적 변수)으로 처리하고, (b) 범주형 변수로 처리하여 모형을 적합할 수 있다. 각 경우에 대해서 처리효과를 분석하고 그 결과를 서로 비교하고 해석하라.

<표 6.10> 콜레스테롤 연구에 관한 연습문제 6.16의 자료

||대조군|대조군|대조군|대조군|처리군|처리군|처리군|처리군|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|초기값|≤3.4|3.4~4.1|4.1~4.9|>4.9|≤3.4|3.4~4.1|4.1~4.9|>4.9|
|≤3.4|18|8|0|0|21|4|2|0|
|3.4~4.1|16|30|13|2|17|25|6|0|
|4.1~4.9|0|14|28|7|11|35|36|6|
|>4.9|0|2|15|22|1|5|14|12|


(a) 적절한 점수를 사용하여 공변량(양적 변수)으로 처리효과를 분석하기

표에서 콜레스테롤 수준은 '≤3.4', '3.4~4.1', '4.1~4.9', '>4.9'로 정의되어 있다.  
이를 양적 변수로 처리하기 위해 우선 '3.4~4.1' 의 값을 3.75로, '4.1~4.9' 의 값을 4.5로 정의하였다.  
'≤3.4'는 3.75와 4.5의 차이인 0.75를 3.75에서 뺀 3.0으로 정의하고,  
'>4.9'는 역시 3.75와 4.5의 차이인 0.75를 4.5에서 더한 5.25로 정의하였다.  

|콜레스테롤 수준|양적 변수|
|:-:|:-:|
|≤3.4|3.0|
|3.4~4.1|3.75|
|4.1~4.9|4.5|
|>4.9|5.25| 

In [78]:
# 표 6.10의 결과를 데이터프레임에 입력
# init: 초기 콜레스테롤 수치, final: 처리후 콜레스테롤 수치, ct: 처리방법(대조군-Con, 처리군-Treat), count: 빈도
data <- data.frame(
  init = rep(c(3.0, 3.75, 4.5, 5.25), times = 8),
  final = rep(rep(c(3.0, 3.75, 4.5, 5.25), each = 4), times = 2),
  ct = rep(c("con", "treat"), each = 16),
  count = as.integer(c(18, 8, 0, 0, 16, 30, 13, 2, 0, 14, 28, 7, 0, 2, 15, 22,
            21, 4, 2, 0, 17, 25, 6, 0, 11, 35, 36, 6, 1, 5, 14, 12))
)

# 데이터 프레임 확인
head(data)


,init,final,ct,count
,<dbl>,<dbl>,<chr>,<int>
1,3.00,3.00,con,18
2,3.75,3.00,con,8
3,4.50,3.00,con,0
4,5.25,3.00,con,0
5,3.00,3.75,con,16
6,3.75,3.75,con,30


In [79]:
# vglm 함수에서 weights 옵션을 사용하기 위해 count=0 인 값은 제외한 data2 생성
data2 = data[data$count != 0,]
head(data2)

,init,final,ct,count
,<dbl>,<dbl>,<chr>,<int>
1,3.00,3.00,con,18
2,3.75,3.00,con,8
5,3.00,3.75,con,16
6,3.75,3.75,con,30
7,4.50,3.75,con,13
8,5.25,3.75,con,2


In [80]:
# 반응변수(Y): 최종 콜레스테롤 수치-final / 예측변수(X): 초기 콜레스테롤 수치-init, 처리방법-ct
fit = vglm(final ~ init+ct, family = cumulative(parallel=TRUE), weights=count, data=data2)
summary(fit)

Call:
vglm(formula = final ~ init + ct, family = cumulative(parallel = TRUE), 
    data = data2, weights = count)

Coefficients: 
              Estimate Std. Error z value Pr(>|z|)    
(Intercept):1   7.4533     0.7065  10.550  < 2e-16 ***
(Intercept):2   9.7463     0.7795  12.503  < 2e-16 ***
(Intercept):3  12.3745     0.8877  13.941  < 2e-16 ***
init           -2.4524     0.1893 -12.952  < 2e-16 ***
cttreat        -0.6045     0.2059  -2.936  0.00332 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Names of linear predictors: logitlink(P[Y<=1]), logitlink(P[Y<=2]), 
logitlink(P[Y<=3])

Residual deviance: 753.0468 on 73 degrees of freedom

Log-likelihood: -376.5234 on 73 degrees of freedom

Number of Fisher scoring iterations: 5 

No Hauck-Donner effect found in any of the estimates


Exponentiated coefficients:
      init    cttreat 
0.08608926 0.54633713 

### 초기 콜레스테롤 수준을 양적 변수로 처리한 비례오즈 누적 로짓모형의 결과 해석

$logit[P(Y\leq j)]=\alpha_j+\beta_1\times init+\beta_2\times ct$ 

1. init: 초기 콜레스테롤 수치
- $\hat\beta_1=-2.4524 (SE=0.1893)$
  - 고정된 j에 대하여 콜레스테롤이 높아질 오즈는 초기 콜레스테롤 값이 1 증가할 때 $exp(-2.4524)=0.086$ 배임
  - 초기 콜레스테롤 수치가 낮을수록 처리 후에 보다 높은 콜레스테롤 수치를 기록하는 경향을 띤다.
    - 다만, 이러한 분석은 신중하게 검토할 필요가 있다.
    - 왜냐하면, 이것은 콜레스테롤 수치를 범주화했기 때문이라고도 해석할 수 있는데, 콜레스테롤 수치가 3.4 이하이면, 처리후에 아무리 콜레스테롤 수치가 낮아지더라도 같은 범주에 속하기 때문이다.
    - '≤3.4'보다 높은 '3.4~4.1', '4.1~4.9', '>4.9' 범주에 속한 경우 처리 이후 더 낮은 범주로 이동할 수 있으나, '≤3.4'는 더 낮은 범주로의 이동이 불가능하다.
    - '>4.9' 범주에 속한 경우 더 높은 범주로 이동할 수 없다.
    - 이러한 사항을 고려할 때, $\hat\beta_1=-2.4524$이라 하더라도 초기 콜레스테롤 수치가 낮을수록 처리 후에 보다 높은 콜레스테롤 수치를 기록하는 경향을 보인다고 단정하기는 어렵다.

2. ct: 처리방법(대조군-con, 처리군-treat)
- $\hat\beta_2=-0.6045 (SE=0.2059)$
  - 처리군(treat)이 더 높은 최종 콜레스테롤 수치를 기록할 오즈는 대조군(con)과 비교하여 $exp(-0.6045)=0.5463$ 배이다.
  - 처리군이 대조군보다 높은 콜레스테롤 수준으로 분류될 오즈가 0.55배라는 것은 처리군은 처리 후에 더 낮은 콜레스테롤 수치를 기록할 경향을 보인다고 할 수 있다.
  - 즉, 사일리움(psyllium)의 섭취가 콜레스테롤 수준을 낮추는데 기여할 수 있다는 의미이다.

In [81]:
# init 모델과의 비교
fit0 = vglm(final ~ init, family = cumulative(parallel=TRUE), weights=count, data=data2)
cat("init 모델과의 비교", "\n")
lrtest(fit, fit0)
# ct 모델과의 비교
fit0 = vglm(final ~ ct, family = cumulative(parallel=TRUE), weights=count, data=data2)
cat("ct 모델과의 비교", "\n")
lrtest(fit, fit0)
# NULL모델과의 비교
fit0 = vglm(final ~ 1, family = cumulative(parallel=TRUE), weights=count, data=data2)
cat("NULL 모델과의 비교", "\n")
lrtest(fit, fit0)

init 모델과의 비교 


Likelihood ratio test

Model 1: final ~ init + ct
Model 2: final ~ init
  #Df  LogLik Df  Chisq Pr(>Chisq)   
1  73 -376.52                        
2  74 -380.91  1 8.7815   0.003043 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

ct 모델과의 비교 


Likelihood ratio test

Model 1: final ~ init + ct
Model 2: final ~ ct
  #Df  LogLik Df  Chisq Pr(>Chisq)    
1  73 -376.52                         
2  74 -489.20  1 225.35  < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

NULL 모델과의 비교 


Likelihood ratio test

Model 1: final ~ init + ct
Model 2: final ~ 1
  #Df  LogLik Df  Chisq Pr(>Chisq)    
1  73 -376.52                         
2  75 -489.53  2 226.01  < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

반응변수(Y)를 최종 콜레스테롤 수치를 정의하고, 예측변수를 init으로 하는 경우, ct로 하는 경우, 1로 하는 경우를 모두 비교해보았다.

모든 경우에서 예측변수를 init + ct 로 한 모델이 유의미한 분석으로 나타났다.

### (2) 범주형 변수로 처리하여 모형 적합하기

양적 변수가 아니라 범주형 변수로 분석하기 위해서는 
1. init 항목을 factor로 변환하고,  
2. vglm 함수에서 family를 cumulative 가 아니라 multinomial 로 바꿔야 한다.

In [82]:
# 반응변수(Y): 최종 콜레스테롤 수치-final / 예측변수(X): 초기 콜레스테롤 수치-init, 처리방법-ct
fit = vglm(final ~ factor(init)+ct, family = multinomial(refLevel=4), weights=count, data=data2)
summary(fit)

Call:
vglm(formula = final ~ factor(init) + ct, family = multinomial(refLevel = 4), 
    data = data2, weights = count)

Coefficients: 
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept):1        4.3354     1.0743   4.036 5.45e-05 ***
(Intercept):2        4.1344     1.0594   3.903 9.52e-05 ***
(Intercept):3        2.1305     1.0770   1.978  0.04791 *  
factor(init)3.75:1  -3.2537     1.1246  -2.893  0.00381 ** 
factor(init)3.75:2  -1.5559     1.0951  -1.421  0.15539    
factor(init)3.75:3  -0.4176     1.1207  -0.373  0.70939    
factor(init)4.5:1   -6.6247     1.2648  -5.238 1.62e-07 ***
factor(init)4.5:2   -4.1892     1.0693  -3.918 8.94e-05 ***
factor(init)4.5:3   -1.5214     1.0718  -1.419  0.15576    
factor(init)5.25:1 -20.8425   453.7135      NA       NA    
factor(init)5.25:2  -6.7238     1.2696  -5.296 1.18e-07 ***
factor(init)5.25:3  -3.2235     1.1017  -2.926  0.00343 ** 
cttreat:1           -1.0074     0.4939  -2.040  0.04137 *  
cttreat:2           -0.9

### 초기 콜레스테롤 수준을 범주형 변수로 처리한 기준범주 로짓모형의 결과 해석

임의로 하나의 기준범주를 선택할 수도 있고, refLevel 옵션을 사용하여 기준범주를 지정할 수도 있다.  
여기서는 refLevel=4를 지정하여, 기본 옵션과 같이 final 범주의 값이 가장 큰 경우를 기준범주로 지정하였다.

기준범주 로짓모형에서 우리가 관심있는 ct(처리군 여부)의 효과를 분석하면 다음과 같다.

- cttreat:1 의 상관계수 = -1.0074
  - 이것은 최종 콜레스테롤 수치의 범주가 1('≥3.4')일 때 최종 콜레스트롤 수치의 범주가 4('<4.9') 일 때와 비교하여
  처리군(treat)이 대조군(con)에 비해서 높은 콜레스테를 수치를 나타낼 오즈가 $exp(-1.0074)=0.365$ 배임을 의미한다.
  - 즉, 최종 콜레스테를 수치가 낮은 경우 (최종 콜레스테롤 수치가 4.9 이상인 때와 비교하여) 사일리움(psyllium)의 섭취가 콜레스테롤 수치를 낮추는 경향이 있음을 의미한다.
  - 이러한 분석은 p값이 0.04137 이므로 유의하다고 볼 수 있다.

- cttreat:2 의 상관계수 = -0.9466
  - 이것은 최종 콜레스테롤 수치의 범주가 2('3.4~4.1')일 때 최종 콜레스트롤 수치의 범주가 4('<4.9') 일 때와 비교하여
  처리군(treat)이 대조군(con)에 비해서 높은 콜레스테를 수치를 나타낼 오즈가 $exp(-0.9466)=0.388$ 배임을 의미한다.
  - 즉, 최종 콜레스테를 수치가 3.4~4.1인 경우 (최종 콜레스테롤 수치가 4.9 이상인 때와 비교하여) 사일리움(psyllium)의 섭취가 콜레스테롤 수치를 낮추는 경향이 있음을 의미한다.
  - 이러한 분석은 p값이 0.01765 이므로 유의하다고 볼 수 있다.

- cttreat:3 의 상관계수 = 0.3349
  - 이것은 최종 콜레스테롤 수치의 범주가 3('4.1~4.9')일 때 최종 콜레스트롤 수치의 범주가 4('<4.9') 일 때와 비교하여
  처리군(treat)이 대조군(con)에 비해서 높은 콜레스테를 수치를 나타낼 오즈가 $exp(0.3349)=1.398$ 배임을 의미한다.
  - 즉, 최종 콜레스테를 수치가 4.1~4.9인 경우 (최종 콜레스테롤 수치가 4.9 이상인 때와 비교하여) 사일리움(psyllium)의 섭취가 콜레스테롤 수치를 높이는 경향이 있음을 의미한다.
  - 다만, 이러한 분석은 p값이 0.31846 이므로 통계적으로 유의하다고 보기 어렵다.